In [40]:
from trueskill import Rating, quality_1vs1, rate_1vs1, quality, rate
alice, bob = Rating(30), Rating(30)  # assign Alice and Bob's ratings
if quality_1vs1(alice, bob) < 0.50:
    print('This match seems to be not so fair')
alice, bob = rate_1vs1(alice, bob)  # update the ratings after the match

This match seems to be not so fair


In [38]:
alice, bob = Rating(100), Rating(99)  # assign Alice and Bob's ratings
quality_1vs1(alice, bob)

0.44592747324992565

In [45]:
r1 = Rating(100)  # 1P's skill
r2 = Rating(1)  # 2P's skill
r3 = Rating(12)  # 3P's skill
t1 = [r1]  # Team A contains just 1P
t2 = [r2, r3]  # Team B contains 2P and 3P

print(t1)
print(t2)

[trueskill.Rating(mu=100.000, sigma=8.333)]
[trueskill.Rating(mu=1.000, sigma=8.333), trueskill.Rating(mu=12.000, sigma=8.333)]


In [46]:
from trueskill import Rating
print('{:.1%} chance to draw'.format(quality([t1, t2])))
(new_r1,), (new_r2, new_r3) = rate([t1, t2], ranks=[1, 0])
print(new_r1)
print(new_r2)
print(new_r3)



0.0% chance to draw
trueskill.Rating(mu=75.814, sigma=7.173)
trueskill.Rating(mu=25.186, sigma=7.173)
trueskill.Rating(mu=36.186, sigma=7.173)


In [56]:
data_url = 'https://course-resources.minerva.kgi.edu/uploaded_files/mke/00090402-4649/tennis-data.csv'
text = requests.get(data_url).content.decode('latin-1').strip()
lines =	(line for line in text.strip().split('\n'))

# Create a TrueSkill environment with 0 probability of a draw
env = trueskill.TrueSkill(draw_probability=0)

players = {}  # player name: player rating
first_game = {}  # player name: date of first game
last_game = {}  # player name: date of last game

count = 0
reader = csv.reader((line.decode('latin-1') for line in requests.get(data_url).iter_lines()))
header = next(reader)  # First line of CSV file is the header
for datum in (dict(zip(header, _)) for _ in reader):
    if datum == {}:
        # Skip empty rows
        continue

    # It turns out names sometimes have trailing spaces, so strip those to avoid duplicates
    winner_name = datum['Winner'].strip()
    loser_name = datum['Loser'].strip()

    # Get or create ratings
    winner = players.get(winner_name) or env.create_rating()
    loser = players.get(loser_name) or env.create_rating()

    # Update ratings, and first and last played dates
    players[winner_name], players[loser_name] = env.rate_1vs1(winner, loser)
    first_game.setdefault(winner_name, datum['Date'])
    first_game.setdefault(loser_name, datum['Date'])
    last_game[winner_name] = last_game[loser_name] = datum['Date']

    # Display progress through data file
    count += 1
    if count % 5000 == 0:
        print(count, 'games processed')
print('done')

5000 games processed
10000 games processed
15000 games processed
20000 games processed
25000 games processed
30000 games processed
35000 games processed
40000 games processed
45000 games processed
done


In [61]:
import numpy as np

In [62]:
# Top 20 players and their skills
leaderboard = sorted(players.items(), key=lambda player: env.expose(player[1]), reverse=True)
for i in range(20):
    player = leaderboard[i]
    print('%2i. %-20s: %.2f ± %.2f [%10s - %10s]' % (
        i+1, player[0], player[1].mu, player[1].sigma,
        first_game[player[0]], last_game[player[0]]))

 1. Djokovic N.         : 41.89 ± 0.97 [20/07/2004 - 11/09/2016]
 2. Murray A.           : 38.65 ± 0.89 [19/04/2005 -  7/09/2016]
 3. Federer R.          : 38.61 ± 0.89 [ 3/01/2000 -  8/07/2016]
 4. Nadal R.            : 37.05 ± 0.88 [15/04/2003 -  4/09/2016]
 5. Nishikori K.        : 35.91 ± 0.85 [17/07/2007 -  9/09/2016]
 6. Wawrinka S.         : 35.62 ± 0.84 [ 7/07/2003 - 25/09/2016]
 7. Soderling R.        : 35.43 ± 0.84 [22/10/2001 - 17/07/2011]
 8. Del Potro J.M.      : 35.34 ± 0.84 [30/01/2006 -  8/09/2016]
 9. Raonic M.           : 35.24 ± 0.84 [12/08/2009 - 22/09/2016]
10. Berdych T.          : 35.21 ± 0.85 [25/08/2003 - 24/09/2016]
11. Agassi A.           : 35.09 ± 0.85 [17/01/2000 -  3/09/2006]
12. Sampras P.          : 34.81 ± 0.89 [17/01/2000 - 26/08/2002]
13. Tsonga J.W.         : 34.62 ± 0.84 [15/09/2004 -  7/09/2016]
14. Rafter P.           : 34.92 ± 0.95 [28/02/2000 - 12/11/2001]
15. Roddick A.          : 34.16 ± 0.84 [28/02/2000 -  5/09/2012]
16. Ferrer D.           :

In [63]:
# This code shows how to access the parameters of the posterior Normal
# distribution over the skill of a particular player.

player_name = 'Federer R.'
player = players[player_name]
print(player_name)
print('mu:', player.mu)
print('sigma:', player.sigma)

Federer R.
mu: 38.61408139627276
sigma: 0.8935283841904839


In [90]:
import scipy.stats as sts

def prob_win(player1, player2):
    one = np.random.normal(player1.mu, player1.sigma, 100000)
    two = np.random.normal(player2.mu, player2.sigma, 100000)
    counter = 0
    for i in range(len(one)):
        if one[i] > two[i]:
            counter += 1
    return counter/100000

In [92]:
player_names = ['Federer R.', 'Nadal R.']
p = prob_win(players[player_names[0]], players[player_names[1]])
print(f'The probability that {player_names[0]} will win against {player_names[1]} is {p:.3f}')

The probability that Federer R. will win against Nadal R. is 0.894
